In [1]:
# Libraries needed: 
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate
from dsgtools.reporting import col_summary
from dsgtools import azure

#### input data

In [2]:
names = ['Data_Record_Identifier', 'Record_Number', 'Business_Name', 'Alternate_Business_Name', 'Primary_Address_Line', 'Primary_City', 
'Primary_State', 'Primary_ZIP_CODE', 'Primary_Business_Phone', 'TaxID', 'Filler_1', 'Secondary_Address_Line_2', 'Secondary_City', 'Secondary_State', 'Secondary_ZIP_CODE', 
'Secondary_Business_Phone', 'Filler_2', 'Owner/Guarantor_Record_ID', 'Owner/Principal_Last_Name', 'Owner/Principal_First_Name', 'Owner/Principal_Middle_Name', 
'Owner/Principal_Address_Line_1', 'Owner/Principal_City', 'Owner/Principal_State', 'Owner/Principal_ZIP_CODE', 'Owner/Principal_Home_Phone', 
'Owner/Principal_SSN', 'Filler_3', 'Permissible_Purpose', 'Filler_4', 'Portfolio_Indicator', ]
specs = [(0, 2), (2, 32), (32, 97), (97, 162), (162, 222), (222, 250), (250, 252), (252, 261), (261, 271), (271, 280), (280, 281), 
(281, 341), (341, 369), (369, 371), (371, 380), (380, 381), (381, 440), (440, 442), (442, 467), (467, 487), (487, 507), (507, 567), 
(567, 595), (595, 597), (597, 606), (606, 616), (616, 625), (625, 626), (626, 628), (628, 629), (629, 637), ]

In [3]:
from dsgtools import azure
input_df = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2447/input.txt", 
                   reader = pd.read_fwf, dtype = str, header = None, names = names, skiprows = [0],
                   colspecs = specs)
print(input_df.shape)

(1912254, 31)


In [4]:
input_df.Record_Number.is_unique

False

In [5]:
pd.set_option('display.max_columns', None)
ids = input_df.Record_Number
input_df[ids.isin(ids[ids.duplicated()])].sort_values("Record_Number").head()

,Data_Record_Identifier,Record_Number,Business_Name,Alternate_Business_Name,Primary_Address_Line,Primary_City,Primary_State,Primary_ZIP_CODE,Primary_Business_Phone,TaxID,Filler_1,Secondary_Address_Line_2,Secondary_City,Secondary_State,Secondary_ZIP_CODE,Secondary_Business_Phone,Filler_2,Owner/Guarantor_Record_ID,Owner/Principal_Last_Name,Owner/Principal_First_Name,Owner/Principal_Middle_Name,Owner/Principal_Address_Line_1,Owner/Principal_City,Owner/Principal_State,Owner/Principal_ZIP_CODE,Owner/Principal_Home_Phone,Owner/Principal_SSN,Filler_3,Permissible_Purpose,Filler_4,Portfolio_Indicator
141034,DD,0000000000000528513,NUNES BROTHERS DAIRY,NaN,20445 ROAD 124,TULARE,CA,93274,NaN,941180527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1,NUNES,FRANK,NaN,1893 E SAN GREGORIO ST,TULARE,CA,93274,NaN,557252458,NaN,NaN,NaN,NON-TSYS
141033,DD,0000000000000528513,NUNES BROTHERS DAIRY,NaN,20445 ROAD 124,TULARE,CA,93274,NaN,941180527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1,NUNES,TONY,L,20429 ROAD 124,TULARE,CA,93274,NaN,557254798,NaN,NaN,NaN,NON-TSYS
1413110,DD,0000000000000559740,"KPADUWA, JULIUS",NaN,2249 INDIAN CREEK RD,DIAMOND BAR,CA,91765,NaN,578780612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1,KPADUWA,STELLA,NaN,NaN,NaN,NaN,NaN,NaN,552993382,NaN,NaN,NaN,NON-TSYS
1413109,DD,0000000000000559740,"KPADUWA, JULIUS",NaN,2249 INDIAN CREEK RD,DIAMOND BAR,CA,91765,NaN,578780612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,954640732,NaN,NaN,NaN,NON-TSYS
120948,DD,0000000000028657963,HUST BROTHERS INC,NaN,712 3RD ST,MARYSVILLE,CA,95901,NaN,941130813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O1,LANZA,DAVID,W,908 WOODRIDGE CT,YUBA CITY,CA,95993,NaN,563330221,NaN,NaN,NaN,NON-TSYS


In [6]:
keep = ["Record_Number", "Business_Name", "Primary_Address_Line", 'Primary_City', 'Primary_State', 'Primary_ZIP_CODE', 'Primary_Business_Phone', 'TaxID',]
new_name = ["Record_Number", "Business_Name", "Business_Address", 'Business_City', 'Business_State', 'Business_Zip', 'Business_Phone', 'Business_TIN',]
input_df = input_df[keep]
input_df.columns = new_name

In [7]:
input_df = input_df.drop_duplicates(subset = ["Record_Number"])
print(input_df.shape)

(1861329, 8)


In [8]:
input_df.to_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2447/input_formated.parquet", 
                    format = '.parquet', overwrite = True)

#### Phone output

In [12]:
from dsgtools import azure
phoneOff = pd.read_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2447/phonesOFF_BANKOFAMERICA.csv', 
                   reader = pd.read_csv, dtype = str, header = None, names = names, usecols = ["UniqueNumber", 'SBFETimeOldest', 'BusinessRecordTimeOldest'])
print(phoneOff.shape)

(1912253, 3)


In [13]:
phoneOff.head()

,UniqueNumber,BusinessRecordTimeOldest,SBFETimeOldest
0,4339931576393847,251,99
1,4339930019570144,221,231
2,7000051009224144891,199,204
3,4003901021312416,184,175
4,5474151341936281,26,9


In [16]:
phoneOff["SBFETimeOldest"] = phoneOff["SBFETimeOldest"].astype(float)
phoneOff["BusinessRecordTimeOldest"] = phoneOff["BusinessRecordTimeOldest"].astype(float)
phoneOff["index_sbfe"] = np.where((phoneOff["SBFETimeOldest"] >=0) & (phoneOff["BusinessRecordTimeOldest"] >= 0), 3, 
                                  np.where((phoneOff["SBFETimeOldest"] >=0) & (phoneOff["BusinessRecordTimeOldest"] < 0), 2, 
                                           np.where((phoneOff["SBFETimeOldest"] <0) & (phoneOff["BusinessRecordTimeOldest"] >= 0), 1, 
                                                    np.where((phoneOff["SBFETimeOldest"] <0) & (phoneOff["BusinessRecordTimeOldest"] < 0), 0, 
                                                             -1)))) # -1 is the NANs
freq(phoneOff["index_sbfe"])

,Count,Pct,Cuml Count,Cuml Pct
index_sbfe,,,,
-1,389.0,0.000203,389.0,0.000203
0,51680.0,0.027026,52069.0,0.027229
1,12367.0,0.006467,64436.0,0.033696
2,328919.0,0.172006,393355.0,0.205702
3,1518898.0,0.794298,1912253.0,1.000000


In [23]:
phoneOff.head()

,UniqueNumberOff,BusinessRecordTimeOldestOff,SBFETimeOldestOff,index_sbfeOff
0,4339931576393847,251.0,99.0,3
1,4339930019570144,221.0,231.0,3
2,7000051009224144891,199.0,204.0,3
3,4003901021312416,184.0,175.0,3
4,5474151341936281,26.0,9.0,3


In [24]:
phoneOff = phoneOff.drop_duplicates(subset = ["UniqueNumberOff"])
print(phoneOff.shape)

(1861328, 4)


In [21]:
from dsgtools import azure
phoneOn = pd.read_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2447/phonesOn_BANKOFAMERICA.csv', 
                   reader = pd.read_csv, dtype = str, header = None, names = names, usecols = ["UniqueNumber", 'SBFETimeOldest', 'BusinessRecordTimeOldest'])
print(phoneOn.shape)
phoneOn.UniqueNumber.is_unique

(1912253, 3)


False

In [26]:
phoneOn = phoneOn.drop_duplicates(subset = ["UniqueNumber"])
print(phoneOn.shape)

(1861328, 3)


In [27]:
phoneOn["SBFETimeOldest"] = phoneOn["SBFETimeOldest"].astype(float)
phoneOn["BusinessRecordTimeOldest"] = phoneOn["BusinessRecordTimeOldest"].astype(float)
phoneOn["index_sbfe"] = np.where((phoneOn["SBFETimeOldest"] >=0) & (phoneOn["BusinessRecordTimeOldest"] >= 0), 3, 
                                  np.where((phoneOn["SBFETimeOldest"] >=0) & (phoneOn["BusinessRecordTimeOldest"] < 0), 2, 
                                           np.where((phoneOn["SBFETimeOldest"] <0) & (phoneOn["BusinessRecordTimeOldest"] >= 0), 1, 
                                                    np.where((phoneOn["SBFETimeOldest"] <0) & (phoneOn["BusinessRecordTimeOldest"] < 0), 0, 
                                                             -1)))) # -1 is the NANs
freq(phoneOn["index_sbfe"])

/tmp/ipykernel_1053/1178612181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phoneOn["SBFETimeOldest"] = phoneOn["SBFETimeOldest"].astype(float)
/tmp/ipykernel_1053/1178612181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phoneOn["BusinessRecordTimeOldest"] = phoneOn["BusinessRecordTimeOldest"].astype(float)
/tmp/ipykernel_1053/1178612181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Count,Pct,Cuml Count,Cuml Pct
index_sbfe,,,,
-1,367.0,0.000197,367.0,0.000197
0,40368.0,0.021688,40735.0,0.021885
1,36989.0,0.019872,77724.0,0.041757
2,129375.0,0.069507,207099.0,0.111264
3,1654229.0,0.888736,1861328.0,1.000000


In [28]:
# phoneOff.columns = [x + "Off" for x in phoneOff.columns]
phoneOn.columns = [x + "On" for x in phoneOn.columns]
merged = phoneOff.merge(phoneOn, left_on = "UniqueNumberOff", right_on = "UniqueNumberOn")
print(merged.shape)

(1861328, 8)


In [38]:
print(merged[merged["index_sbfeOn"]!= merged["index_sbfeOff"]].shape)
freq("index_sbfeOn", "index_sbfeOff", df = merged, cross = True, observed= True, with_stats=False)

(239023, 8)


index_sbfeOff                                       
                        -1        0        1         2          3
                     Count    Count    Count     Count      Count
index_sbfeOn                                                     
3                      0.0   7823.0    564.0  188647.0  1457195.0
2                      0.0    536.0     70.0  126740.0     2029.0
0                      0.0  27821.0    234.0    7345.0     4968.0
1                      0.0  14285.0  10182.0    4189.0     8333.0
-1                   367.0      0.0      0.0       0.0        0.0

In [33]:
temp = merged[merged["index_sbfeOn"]!= merged["index_sbfeOff"]]
print(temp.shape)
input_subset = input_df.merge(temp, left_on = "Record_Number", right_on = "UniqueNumberOff")
print(input_subset.shape)

(239023, 8)
(239023, 16)


In [39]:
input_subset

,Record_Number,Business_Name,Business_Address,Business_City,Business_State,Business_Zip,Business_Phone,Business_TIN,UniqueNumberOff,BusinessRecordTimeOldestOff,SBFETimeOldestOff,index_sbfeOff,UniqueNumberOn,BusinessRecordTimeOldestOn,SBFETimeOldestOn,index_sbfeOn
0,5474152218316813,WEIGUANG HUANG CONSTRUCTION,9333 GUESS ST,ROSEMEAD,CA,917701907,6266791788,.,5474152218316813,-1.0,80.0,2,5474152218316813,24.0,80.0,3
1,4192000003716784,WAYNE GONZALEZ ROOFING CONTRACTOR I,5625 PATTERSON RD,RIVERVIEW,FL,335783729,8139179246,.,4192000003716784,2.0,-99.0,1,4192000003716784,-1.0,189.0,2
2,5474151417074108,JOSE I LINARES,3814 CAROLYN AVE,FAIRFAX,VA,220313520,5713147537,.,5474151417074108,-1.0,28.0,2,5474151417074108,20.0,27.0,3
3,5474151502991984,BACK YARD PIZZA LLC,7129 S PLACITA ATOYAC,TUCSON,AZ,857467994,5408367724,.,5474151502991984,1.0,2.0,3,5474151502991984,-1.0,-99.0,0
4,5474151366024716,"ARTHUR W. LEACH, ATTORNEY AT LAW",115 ARDSLEY LN,ALPHARETTA,GA,300058607,4047866443,.,5474151366024716,161.0,230.0,3,5474151366024716,39.0,-99.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239018,5474151666816936,CLEANING 2020 INC,11185 NW 7TH ST APT 1,MIAMI,FL,331723690,7868570790,.,5474151666816936,-1.0,7.0,2,5474151666816936,24.0,6.0,3
239019,4339931802680074,SI THONG INC,9898 RARITAN AVE,FOUNTAIN VALLEY,CA,927081047,7147242042,.,4339931802680074,-1.0,142.0,2,4339931802680074,79.0,142.0,3
239020,5474151697505086,RR GAMING LLC,1730 SUMMIT GLEN LN NE,ATLANTA,GA,303292594,8652585755,.,5474151697505086,-1.0,12.0,2,5474151697505086,15.0,11.0,3
239021,5474152175552491,AREPA 69 CORP,4401 NW 87TH AVE UNIT 617,DORAL,FL,331782794,7863108908,.,5474152175552491,-1.0,2.0,2,5474152175552491,-1.0,-99.0,0


In [35]:
input_subset[['Record_Number', 'Business_Name', 'Business_Address', 'Business_City',
       'Business_State', 'Business_Zip', 'Business_Phone', 'Business_TIN',
       'index_sbfeOff', 'index_sbfeOn']].to_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/ARMBS_2447/filtered_df_with_sbfe_index_changes.csv", 
                    format = '.csv', overwrite = True, index=False)

In [10]:
names = ['UniqueNumber', 
'LEX ID', 
'Placeholder/Filler', 
'CompanyName', 
'StreetAddress1', 
'City', 
'State', 
'Zip5', 
'VerificationBusInputName', 
'VerificationBusInputAddr', 
'VerificationBusInputPhone', 
'VerificationBusInputFEIN', 
'VerificationBusInputIndustry', 
'BusinessRecordTimeOldest', 
'BusinessRecordTimeNewest', 
'BusinessRecordUpdated12Month', 
'BusinessActivity03Month', 
'BusinessActivity06Month', 
'BusinessActivity12Month', 
'BusinessAddrCount', 
'FirmAgeEstablished', 
'FirmSICCode', 
'FirmNAICSCode', 
'FirmEmployeeCount', 
'FirmReportedSales', 
'FirmReportedEarnings', 
'FirmIRSRetirementPlan', 
'FirmNonProfit', 
'OrgLocationCount', 
'OrgRelatedCount', 
'OrgParentCompany', 
'OrgLegalEntityCount', 
'OrgAddrLegalEntityCount', 
'OrgSingleLocation', 
'SOSTimeIncorporation', 
'SOSTimeAgentChange', 
'SOSEverDefunct', 
'SOSStateCount', 
'SOSForeignStateFlag', 
'BankruptcyCount', 
'BankruptcyCount12Month', 
'BankruptcyCount24Month', 
'BankruptcyChapter', 
'BankruptcyTimeNewest', 
'LienCount', 
'LienCount12Month', 
'LienCount24Month', 
'LienType', 
'LienTimeNewest', 
'LienTimeOldest', 
'LienDollarTotal', 
'JudgmentCount', 
'JudgmentCount12Month', 
'JudgmentCount24Month', 
'JudgmentType', 
'JudgmentTimeNewest', 
'JudgmentTimeOldest', 
'JudgmentDollarTotal', 
'LienJudgmentDollarTotal', 
'AssetPropertyCount', 
'AssetPropertyStateCount', 
'AssetPropertyLotSizeTotal', 
'AssetPropertyAssessedTotal', 
'AssetPropertySqFootageTotal', 
'AssetAircraftCount', 
'AssetWatercraftCount', 
'UCCCount', 
'UCCTimeNewest', 
'UCCTimeOldest', 
'GovernmentDebarred', 
'InquiryHighRisk12Month', 
'InquiryHighRisk03Month', 
'InquiryCredit12Month', 
'InquiryCredit03Month', 
'Inquiry12Month', 
'Inquiry03Month', 
'InquiryConsumerAddress', 
'InquiryConsumerPhone', 
'InquiryConsumerAddressSSN', 
'BusExecLinkAuthRepNameOnFile', 
'BusExecLinkAuthRepAddrOnFile', 
'BusExecLinkAuthRepSSNOnFile', 
'BusExecLinkAuthRepPhoneOnFile', 
'BusExecLinkBusNameAuthRepFirst', 
'BusExecLinkBusNameAuthRepLast', 
'BusExecLinkBusNameAuthRepFull', 
'BusExecLinkAuthRepSSNBusFEIN', 
'BusExecLinkPropertyOverlapCount', 
'BusExecLinkBusAddrAuthRepOwned', 
'BusExecLinkUtilityOverlapCount', 
'BusExecLinkInquiryOverlapCount', 
'BusExecLinkAuthRepAddrBusAddr', 
'BusExecLinkAuthRepPhoneBusPhone', 
'BusExecLinkAuthRep2NameOnFile', 
'BusExecLinkAuthRep2AddrOnFile', 
'BusExecLinkAuthRep2PhoneOnFile', 
'BusExecLinkAuthRep2SSNOnFile', 
'BusExecLinkBusNameAuthRep2First', 
'BusExecLinkBusNameAuthRep2Last', 
'BusExecLinkBusNameAuthRep2Full', 
'BusExecLinkAuthRep2SSNBusFEIN', 
'BusExecLinkPropertyOverlapCount2', 
'BusExecLinkBusAddrAuthRep2Owned', 
'BusExecLinkUtilityOverlapCount2', 
'BusExecLinkInquiryOverlapCount2', 
'BusExecLinkAuthRep2AddrBusAddr', 
'BusExecLinkAuthRep2PhoneBusPhone', 
'BusExecLinkAuthRep3NameOnFile', 
'BusExecLinkAuthRep3AddrOnFile', 
'BusExecLinkAuthRep3PhoneOnFile', 
'BusExecLinkAuthRep3SSNOnFile', 
'BusExecLinkBusNameAuthRep3First', 
'BusExecLinkBusNameAuthRep3Last', 
'BusExecLinkBusNameAuthRep3Full', 
'BusExecLinkAuthRep3SSNBusFein', 
'BusExecLinkPropertyOverlapCount3', 
'BusExecLinkBusAddrAuthRep3Owned', 
'BusExecLinkUtilityOverlapCount3', 
'BusExecLinkInquiryOverlapCount3', 
'BusExecLinkAuthRep3AddrBusAddr', 
'BusExecLinkAuthRep3PhoneBusPhone', 
'BusFEINPersonOverlap', 
'BusFEINPersonAddrOverlap', 
'BusFEINPersonPhoneOverlap', 
'BusAddrPersonNameOverlap', 
'InputAddrConsumerCount', 
'InputAddrSourceCount', 
'InputAddrType', 
'InputAddrBusinessOwned', 
'InputAddrLotSize', 
'InputAddrAssessedTotal', 
'InputAddrSqFootage', 
'InputPhoneProblems', 
'InputPhoneEntityCount', 
'InputPhoneMobile', 
'AssociateCount', 
'AssociateHighCrimeAddrCount', 
'AssociateFelonyCount', 
'AssociateCountWithFelony', 
'AssociateBankruptCount', 
'AssociateCountWithBankruptcy', 
'AssociateBankrupt1YearCount', 
'AssociateLienCount', 
'AssociateCountWithLien', 
'AssociateJudgmentCount', 
'AssociateCountWithJudgment', 
'AssociateHighRiskAddrCount', 
'AssociateWatchlistCount', 
'AssociateBusinessCount', 
'AssociateCityCount', 
'AssociateCountyCount', 
'SBFEVerBusName', 
'SBFEVerBusAddr', 
'SBFEVerBusPhone', 
'SBFEVerBusIndustryCode', 
'SBFEBusExecLinkRep1Name', 
'SBFEBusExecLinkRep1Addr', 
'SBFEBusExecLinkRep1Phone', 
'SBFEBusExecLinkRep1SSN', 
'SBFEBusExecLinkRep2Name', 
'SBFEBusExecLinkRep2Addr', 
'SBFEBusExecLinkRep2Phone', 
'SBFEBusExecLinkRep2SSN', 
'SBFEBusExecLinkRep3Name', 
'SBFEBusExecLinkRep3Addr', 
'SBFEBusExecLinkRep3Phone', 
'SBFEBusExecLinkRep3SSN', 
'SBFETimeOldestCycle', 
'SBFETimeNewestCycle', 
'SBFEAccountCount', 
'SBFEOpenCount06M', 
'SBFEOpenCount12M', 
'SBFEOpenCount24M', 
'SBFEOpenCount36M', 
'SBFEOpenCount60M', 
'SBFETimeOldest', 
'SBFETimeNewest', 
'SBFETimeNewestClosed', 
'SBFELoanCount', 
'SBFETimeOldestLoan', 
'SBFETimeNewestLoan', 
'SBFETimeNewestClosedLoan', 
'SBFELineCount', 
'SBFETimeOldestLine', 
'SBFETimeNewestLine', 
'SBFETimeNewestClosedLine', 
'SBFECardCount', 
'SBFETimeOldestCard', 
'SBFETimeNewestCard', 
'SBFETimeNewestClosedCard', 
'SBFELeaseCount', 
'SBFETimeOldestLease', 
'SBFETimeNewestLease', 
'SBFETimeNewestClosedLease', 
'SBFELetterCount', 
'SBFETimeOldestLetter', 
'SBFETimeNewestLetter', 
'SBFETimeNewestClosedLetter', 
'SBFEOELineCount', 
'SBFETimeOldestOELine', 
'SBFETimeNewestOELine', 
'SBFETimeNewestClosedOELine', 
'SBFEOtherCount', 
'SBFETimeOldestOther', 
'SBFETimeNewestOther', 
'SBFETimeNewestClosedOther', 
'SBFEOpenCount', 
'SBFEOpenLoanCount', 
'SBFEOpenLineCount', 
'SBFEOpenCardCount', 
'SBFEOpenLeaseCount', 
'SBFEOpenLetterCount', 
'SBFEOpenOELineCount', 
'SBFEOpenOtherCount', 
'SBFEOpenTypeCount', 
'SBFEOpenCardOnly', 
'SBFEActiveCount', 
'SBFEActiveLoanCount', 
'SBFEActiveLineCount', 
'SBFEActiveCardCount', 
'SBFEActiveLeaseCount', 
'SBFEActiveLetterCount', 
'SBFEActiveOELineCount', 
'SBFEActiveOtherCount', 
'SBFEClosedCount', 
'SBFEClosedLoanCount', 
'SBFEClosedLineCount', 
'SBFEClosedCardCount', 
'SBFEClosedLeaseCount', 
'SBFEClosedLetterCount', 
'SBFEClosedOELineCount', 
'SBFEClosedOtherCount', 
'SBFEClosedInvoluntaryCount', 
'SBFEClosedVoluntaryCount', 
'SBFEStaleClosedCount', 
'SBFEStaleClosedLoanCount', 
'SBFEStaleClosedLineCount', 
'SBFEStaleClosedCardCount', 
'SBFEStaleClosedLeaseCount', 
'SBFEStaleClosedLetterCount', 
'SBFEStaleClosedOELineCount', 
'SBFEStaleClosedOtherCount', 
'SBFEInactiveCount', 
'SBFEInactiveLoanCount', 
'SBFEInactiveLineCount', 
'SBFEInactiveCardCount', 
'SBFEInactiveLeaseCount', 
'SBFEInactiveLetterCount', 
'SBFEInactiveOELineCount', 
'SBFEInactiveOtherCount', 
'SBFERecentBalanceAmt', 
'SBFERecentBalanceLoanAmt', 
'SBFERecentBalanceLineAmt', 
'SBFERecentBalanceCardAmt', 
'SBFERecentBalanceLeaseAmt', 
'SBFERecentBalanceLetterAmt', 
'SBFERecentBalanceOELineAmt', 
'SBFERecentBalanceOtherAmt', 
'SBFEAvgBalance06M', 
'SBFEAvgBalance12M', 
'SBFEAvgBalance24M', 
'SBFEAvgBalance36M', 
'SBFEAvgBalance60M', 
'SBFEAvgBalance84M', 
'SBFEAvgBalanceEver', 
'SBFEAvgBalanceLoan06M', 
'SBFEAvgBalanceLoan12M', 
'SBFEAvgBalanceLoan24M', 
'SBFEAvgBalanceLoan36M', 
'SBFEAvgBalanceLoan60M', 
'SBFEAvgBalanceLoan84M', 
'SBFEAvgBalanceLoanEver', 
'SBFEAvgBalanceLine06M', 
'SBFEAvgBalanceLine12M', 
'SBFEAvgBalanceLine24M', 
'SBFEAvgBalanceLine36M', 
'SBFEAvgBalanceLine60M', 
'SBFEAvgBalanceLine84M', 
'SBFEAvgBalanceLineEver', 
'SBFEAvgBalanceCard06M', 
'SBFEAvgBalanceCard12M', 
'SBFEAvgBalanceCard24M', 
'SBFEAvgBalanceCard36M', 
'SBFEAvgBalanceCard60M', 
'SBFEAvgBalanceCard84M', 
'SBFEAvgBalanceCardEver', 
'SBFEAvgBalanceLease06M', 
'SBFEAvgBalanceLease12M', 
'SBFEAvgBalanceLease24M', 
'SBFEAvgBalanceLease36M', 
'SBFEAvgBalanceLease60M', 
'SBFEAvgBalanceLease84M', 
'SBFEAvgBalanceLeaseEver', 
'SBFEAvgBalanceLetter06M', 
'SBFEAvgBalanceLetter12M', 
'SBFEAvgBalanceLetter24M', 
'SBFEAvgBalanceLetter36M', 
'SBFEAvgBalanceLetter60M', 
'SBFEAvgBalanceLetter84M', 
'SBFEAvgBalanceLetterEver', 
'SBFEAvgBalanceOELine06M', 
'SBFEAvgBalanceOELine12M', 
'SBFEAvgBalanceOELine24M', 
'SBFEAvgBalanceOELine36M', 
'SBFEAvgBalanceOELine60M', 
'SBFEAvgBalanceOELine84M', 
'SBFEAvgBalanceOELineEver', 
'SBFEAvgBalanceOther06M', 
'SBFEAvgBalanceOther12M', 
'SBFEAvgBalanceOther24M', 
'SBFEAvgBalanceOther36M', 
'SBFEAvgBalanceOther60M', 
'SBFEAvgBalanceOther84M', 
'SBFEAvgBalanceOtherEver', 
'SBFEBalanceCount', 
'SBFEBalanceCount06M', 
'SBFEBalanceCount12M', 
'SBFEBalanceCount24M', 
'SBFEBalanceCount36M', 
'SBFEBalanceCount60M', 
'SBFEBalanceCount84M', 
'SBFEBalanceCountEver', 
'SBFEBalanceLoanCount', 
'SBFEBalanceLoanCount06M', 
'SBFEBalanceLoanCount12M', 
'SBFEBalanceLoanCount24M', 
'SBFEBalanceLoanCount36M', 
'SBFEBalanceLoanCount60M', 
'SBFEBalanceLoanCount84M', 
'SBFEBalanceLoanCountEver', 
'SBFEBalanceLineCount', 
'SBFEBalanceLineCount06M', 
'SBFEBalanceLineCount12M', 
'SBFEBalanceLineCount24M', 
'SBFEBalanceLineCount36M', 
'SBFEBalanceLineCount60M', 
'SBFEBalanceLineCount84M', 
'SBFEBalanceLineCountEver', 
'SBFEBalanceCardCount', 
'SBFEBalanceCardCount06M', 
'SBFEBalanceCardCount12M', 
'SBFEBalanceCardCount24M', 
'SBFEBalanceCardCount36M', 
'SBFEBalanceCardCount60M', 
'SBFEBalanceCardCount84M', 
'SBFEBalanceCardCountEver', 
'SBFEBalanceLeaseCount', 
'SBFEBalanceLeaseCount06M', 
'SBFEBalanceLeaseCount12M', 
'SBFEBalanceLeaseCount24M', 
'SBFEBalanceLeaseCount36M', 
'SBFEBalanceLeaseCount60M', 
'SBFEBalanceLeaseCount84M', 
'SBFEBalanceLeaseCountEver', 
'SBFEBalanceLetterCount', 
'SBFEBalanceLetterCount06M', 
'SBFEBalanceLetterCount12M', 
'SBFEBalanceLetterCount24M', 
'SBFEBalanceLetterCount36M', 
'SBFEBalanceLetterCount60M', 
'SBFEBalanceLetterCount84M', 
'SBFEBalanceLetterCountEver', 
'SBFEBalanceOELineCount', 
'SBFEBalanceOELineCount06M', 
'SBFEBalanceOELineCount12M', 
'SBFEBalanceOELineCount24M', 
'SBFEBalanceOELineCount36M', 
'SBFEBalanceOELineCount60M', 
'SBFEBalanceOELineCount84M', 
'SBFEBalanceOELineCountEver', 
'SBFEBalanceOtherCount', 
'SBFEBalanceOtherCount06M', 
'SBFEBalanceOtherCount12M', 
'SBFEBalanceOtherCount24M', 
'SBFEBalanceOtherCount36M', 
'SBFEBalanceOtherCount60M', 
'SBFEBalanceOtherCount84M', 
'SBFEBalanceOtherCountEver', 
'SBFEBalanceClosedCount', 
'SBFEBalanceClosedCount12M', 
'SBFEOriginalLimitInstallmentAmt', 
'SBFEOriginalLimitLoanAmt', 
'SBFEOriginalLimitLeaseAmt', 
'SBFECurrentLimitRevolvingAmt', 
'SBFECurrentLimitLineAmt', 
'SBFECurrentLimitCardAmt', 
'SBFECurrentLimitOELineAmt', 
'SBFEPaymentDueAmt', 
'SBFEPaymentDueLoanAmt', 
'SBFEPaymentDueLineAmt', 
'SBFEPaymentDueCardAmt', 
'SBFEPaymentDueLeaseAmt', 
'SBFEPaymentDueLetterAmt', 
'SBFEPaymentDueOELineAmt', 
'SBFEPaymentDueOtherAmt', 
'SBFELastPaymentAmt', 
'SBFELastPaymentLoanAmt', 
'SBFELastPaymentLineAmt', 
'SBFELastPaymentCardAmt', 
'SBFELastPaymentLeaseAmt', 
'SBFELastPaymentLetterAmt', 
'SBFELastPaymentOELineAmt', 
'SBFELastPaymentOtherAmt', 
'SBFEUtilCurrentRevolving', 
'SBFEUtilCurrentLine', 
'SBFEUtilCurrentCard', 
'SBFEUtilCurrentOELine', 
'SBFEUtil75RevolvingCount', 
'SBFEUtil75LineCount', 
'SBFEUtil75CardCount', 
'SBFEUtil75OELineCount', 
'SBFEUtil30RevolvingCount', 
'SBFEUtil30LineCount', 
'SBFEUtil30CardCount', 
'SBFEUtil30OELineCount', 
'SBFEAvgUtil06M', 
'SBFEAvgUtil12M', 
'SBFEAvgUtil24M', 
'SBFEAvgUtil36M', 
'SBFEAvgUtil60M', 
'SBFEAvgUtil84M', 
'SBFEAvgUtilEver', 
'SBFEAvgUtilLine06M', 
'SBFEAvgUtilLine12M', 
'SBFEAvgUtilLine24M', 
'SBFEAvgUtilLine36M', 
'SBFEAvgUtilLine60M', 
'SBFEAvgUtilLine84M', 
'SBFEAvgUtilLineEver', 
'SBFEAvgUtilCard06M', 
'SBFEAvgUtilCard12M', 
'SBFEAvgUtilCard24M', 
'SBFEAvgUtilCard36M', 
'SBFEAvgUtilCard60M', 
'SBFEAvgUtilCard84M', 
'SBFEAvgUtilCardEver', 
'SBFEAvgUtilOELine06M', 
'SBFEAvgUtilOELine12M', 
'SBFEAvgUtilOELine24M', 
'SBFEAvgUtilOELine36M', 
'SBFEAvgUtilOELine60M', 
'SBFEAvgUtilOELine84M', 
'SBFEAvgUtilOELineEver', 
'SBFEHighUtilRevolvingAmt', 
'SBFEHighUtilLineAmt', 
'SBFEHighUtilCardAmt', 
'SBFEHighUtiliztionOELineAmt', 
'SBFEHighDelqOpen', 
'SBFEHighDelq06M', 
'SBFEHighDelq12M', 
'SBFEHighDelq24M', 
'SBFEHighDelq36M', 
'SBFEHighDelq60M', 
'SBFEHighDelq84M', 
'SBFEHighDelqEver', 
'SBFEDelqLoan', 
'SBFEHighDelqLoan06M', 
'SBFEHighDelqLoan12M', 
'SBFEHighDelqLoan24M', 
'SBFEHighDelqLoan36M', 
'SBFEHighDelqLoan60M', 
'SBFEHighDelqLoan84M', 
'SBFEHighDelqLoanEver', 
'SBFEHighDelqLine', 
'SBFEHighDelqLine06M', 
'SBFEHighDelqLine12M', 
'SBFEHighDelqLine24M', 
'SBFEHighDelqLine36M', 
'SBFEHighDelqLine60M', 
'SBFEHighDelqLine84M', 
'SBFEHighDelqLineEver', 
'SBFEHighDelqCard', 
'SBFEHighDelqCard06M', 
'SBFEHighDelqCard12M', 
'SBFEHighDelqCard24M', 
'SBFEHighDelqCard36M', 
'SBFEHighDelqCard60M', 
'SBFEHighDelqCard84M', 
'SBFEHighDelqCardEver', 
'SBFEHighDelqLease', 
'SBFEHighDelqLease06M', 
'SBFEHighDelqLease12', 
'SBFEHighDelqLease24M', 
'SBFEHighDelqLease36M', 
'SBFEHighDelqLease60M', 
'SBFEHighDelqLease84M', 
'SBFEHighDelqLeaseEver', 
'SBFEHighDelqLetter', 
'SBFEHighDelqLetter06M', 
'SBFEHighDelqLetter12M', 
'SBFEHighDelqLetter24M', 
'SBFEHighDelqLetter36M', 
'SBFEHighDelqLetter60M', 
'SBFEHighDelqLetter84M', 
'SBFEHighDelqLetterEver', 
'SBFEHighDelqOELine', 
'SBFEHighDelqOELine06M', 
'SBFEHighDelqOELine12M', 
'SBFEHighDelqOELine24M', 
'SBFEHighDelqOELine36M', 
'SBFEHighDelqOELine60M', 
'SBFEHighDelqOELine84M', 
'SBFEHighDelqOELineEver', 
'SBFEHighDelqother', 
'SBFEHighDelqother06M', 
'SBFEHighDelqother12M', 
'SBFEHighDelqother24M', 
'SBFEHighDelqother36M', 
'SBFEHighDelqother60M', 
'SBFEHighDelqother84M', 
'SBFEHighDelqotherEver', 
'SBFESatisfactoryCount', 
'SBFESatisfactoryLoanCount', 
'SBFESatisfactoryLineCount', 
'SBFESatisfactoryCardCount', 
'SBFESatisfactoryleaseCount', 
'SBFESatisfactoryLetterCount', 
'SBFESatisfactoryOELineCount', 
'SBFESatisfactoryOtherCount', 
'SBFETimeNewestDelq', 
'SBFEDelq1Count', 
'SBFEDelq1Count06M', 
'SBFEDelq1Count12M', 
'SBFEDelq1Count24M', 
'SBFEDelq1Count36', 
'SBFEDelq1Count60', 
'SBFEDelq1Count84M', 
'SBFEDelq1CountEver', 
'SBFEDelq31Count', 
'SBFEDelq31Count06M', 
'SBFEDelq31Count12M', 
'SBFEDelq31Count24M', 
'SBFEDelq31Count36M', 
'SBFEDelq31Count60M', 
'SBFEDelq31Count84M', 
'SBFEDelq31CountEver', 
'SBFEDelq31LoanCount', 
'SBFEDelq31LoanCount06M', 
'SBFEDelq31LoanCount12M', 
'SBFEDelq31LoanCount24M', 
'SBFEDelq31LoanCount36M', 
'SBFEDelq31LoanCount60M', 
'SBFEDelq31LoanCount84M', 
'SBFEDelq31LoanCountEver', 
'SBFEDelq31LineCount', 
'SBFEDelq31LineCount06M', 
'SBFEDelq31LineCount12M', 
'SBFEDelq31LineCount24M', 
'SBFEDelq31LineCount36M', 
'SBFEDelq31LineCount60M', 
'SBFEDelq31LineCount84M', 
'SBFEDelq31LineCountEver', 
'SBFEDelq31CardCount', 
'SBFEDelq31CardCount06M', 
'SBFEDelq31CardCount12M', 
'SBFEDelq31CardCount24M', 
'SBFEDelq31CardCount36M', 
'SBFEDelq31CardCount60M', 
'SBFEDelq31CardCount84M', 
'SBFEDelq31CardCountEver', 
'SBFEDelq31LeaseCount', 
'SBFEDelq31LeaseCount06M', 
'SBFEDelq31LeaseCount12M', 
'SBFEDelq31LeaseCount24M', 
'SBFEDelq31LeaseCount36M', 
'SBFEDelq31LeaseCount60M', 
'SBFEDelq31LeaseCount84M', 
'SBFEDelq31LeaseCountEver', 
'SBFEDelq31LetterCount', 
'SBFEDelq31LetterCount06M', 
'SBFEDelq31LetterCount12M', 
'SBFEDelq31LetterCount24M', 
'SBFEDelq31LetterCount36M', 
'SBFEDelq31LetterCount60M', 
'SBFEDelq31LetterCount84M', 
'SBFEDelq31LetterCountEver', 
'SBFEDelq31OELineCount', 
'SBFEDelq31OELineCount06M', 
'SBFEDelq31OELineCount12M', 
'SBFEDelq31OELineCount24M', 
'SBFEDelq31OELineCount36M', 
'SBFEDelq31OELineCount60M', 
'SBFEDelq31OELineCount84M', 
'SBFEDelq31OELineCountEver', 
'SBFEDelq31OtherCount', 
'SBFEDelq31OtherCount06M', 
'SBFEDelq31OtherCount12M', 
'SBFEDelq31OtherCount24M', 
'SBFEDelq31OtherCount36M', 
'SBFEDelq31OtherCount60M', 
'SBFEDelq31OtherCount84M', 
'SBFEDelq31OtherCountEver', 
'SBFEDelq61Count', 
'SBFEDelq61Count06M', 
'SBFEDelq61Count12M', 
'SBFEDelq61Count24M', 
'SBFEDelq61Count36M', 
'SBFEDelq61Count60M', 
'SBFEDelq61Count84M', 
'SBFEDelq61CountEver', 
'SBFEDelq61LoanCount', 
'SBFEDelq61LoanCount06M', 
'SBFEDelq61LoanCount12M', 
'SBFEDelq61LoanCount24M', 
'SBFEDelq61LoanCount36M', 
'SBFEDelq61LoanCount60M', 
'SBFEDelq61LoanCount84M', 
'SBFEDelq61LoanCountEver', 
'SBFEDelq61LineCount', 
'SBFEDelq61LineCount06M', 
'SBFEDelq61LineCount12M', 
'SBFEDelq61LineCount24M', 
'SBFEDelq61LineCount36M', 
'SBFEDelq61LineCount60M', 
'SBFEDelq61LineCount84M', 
'SBFEDelq61LineCountEver', 
'SBFEDelq61CardCount', 
'SBFEDelq61CardCount06M', 
'SBFEDelq61CardCount12M', 
'SBFEDelq61CardCount24M', 
'SBFEDelq61CardCount36M', 
'SBFEDelq61CardCount60M', 
'SBFEDelq61CardCount84M', 
'SBFEDelq61CardCountEver', 
'SBFEDelq61LeaseCount', 
'SBFEDelq61LeaseCount06M', 
'SBFEDelq61LeaseCount12M', 
'SBFEDelq61LeaseCount24M', 
'SBFEDelq61LeaseCount36M', 
'SBFEDelq61LeaseCount60M', 
'SBFEDelq61LeaseCount84M', 
'SBFEDelq61LeaseCountEver', 
'SBFEDelq61LetterCount', 
'SBFEDelq61LetterCount06M', 
'SBFEDelq61LetterCount12M', 
'SBFEDelq61LetterCount24M', 
'SBFEDelq61LetterCount36M', 
'SBFEDelq61LetterCount60M', 
'SBFEDelq61LetterCount84M', 
'SBFEDelq61LetterCountEver', 
'SBFEDelq61OELineCount', 
'SBFEDelq61OELineCount06M', 
'SBFEDelq61OELineCount12M', 
'SBFEDelq61OELineCount24M', 
'SBFEDelq61OELineCount36M', 
'SBFEDelq61OELineCount60M', 
'SBFEDelq61OELineCount84M', 
'SBFEDelq61OELineCountEver', 
'SBFEDelq61OtherCount', 
'SBFEDelq61OtherCount06M', 
'SBFEDelq61OtherCount12M', 
'SBFEDelq61OtherCount24M', 
'SBFEDelq61OtherCount36M', 
'SBFEDelq61OtherCount60M', 
'SBFEDelq61OtherCount84M', 
'SBFEDelq61OtherCountEver', 
'SBFEDelq91Count', 
'SBFEDelq91Count06M', 
'SBFEDelq91Count12M', 
'SBFEDelq91Count24M', 
'SBFEDelq91Count36M', 
'SBFEDelq91Count60M', 
'SBFEDelq91Count84M', 
'SBFEDelq91CountEver', 
'SBFEDelq91LoanCount', 
'SBFEDelq91LoanCount06M', 
'SBFEDelq91LoanCount12M', 
'SBFEDelq91LoanCount24M', 
'SBFEDelq91LoanCount36M', 
'SBFEDelq91LoanCount60M', 
'SBFEDelq91LoanCount84M', 
'SBFEDelq91LoanCountEver', 
'SBFEDelq91LineCount', 
'SBFEDelq91LineCount06M', 
'SBFEDelq91LineCount12M', 
'SBFEDelq91LineCount24M', 
'SBFEDelq91LineCount36M', 
'SBFEDelq91LineCount60M', 
'SBFEDelq91LineCount84M', 
'SBFEDelq91LineCountEver', 
'SBFEDelq91CardCount', 
'SBFEDelq91CardCount06M', 
'SBFEDelq91CardCount12M', 
'SBFEDelq91CardCount24M', 
'SBFEDelq91CardCount36M', 
'SBFEDelq91CardCount60M', 
'SBFEDelq91CardCount84M', 
'SBFEDelq91CardCountEver', 
'SBFEDelq91LeaseCount', 
'SBFEDelq91LeaseCount06M', 
'SBFEDelq91LeaseCount12M', 
'SBFEDelq91LeaseCount24M', 
'SBFEDelq91LeaseCount36M', 
'SBFEDelq91LeaseCount60M', 
'SBFEDelq91LeaseCount84M', 
'SBFEDelq91LeaseCountEver', 
'SBFEDelq91LetterCount', 
'SBFEDelq91LetterCount06M', 
'SBFEDelq91LetterCount12M', 
'SBFEDelq91LetterCount24M', 
'SBFEDelq91LetterCount36M', 
'SBFEDelq91LetterCount60M', 
'SBFEDelq91LetterCount84M', 
'SBFEDelq91LetterCountEver', 
'SBFEDelq91OELineCount', 
'SBFEDelq91OELineCount06M', 
'SBFEDelq91OELineCount12M', 
'SBFEDelq91OELineCount24M', 
'SBFEDelq91OELineCount36M', 
'SBFEDelq91OELineCount60M', 
'SBFEDelq91OELineCount84M', 
'SBFEDelq91OELineCountEver', 
'SBFEDelq91OtherCount', 
'SBFEDelq91OtherCount06M', 
'SBFEDelq91OtherCount12M', 
'SBFEDelq91OtherCount24M', 
'SBFEDelq91OtherCount36M', 
'SBFEDelq91OtherCount60M', 
'SBFEDelq91OtherCount84M', 
'SBFEDelq91OtherCountEver', 
'SBFEDelq121Count', 
'SBFEDelq121Count06M', 
'SBFEDelq121Count12M', 
'SBFEDelq121Count24M', 
'SBFEDelq121Count36M', 
'SBFEDelq121Count60M', 
'SBFEDelq121Count84M', 
'SBFEDelq121CountEver', 
'SBFEDelq121LoanCount', 
'SBFEDelq121LoanCount06M', 
'SBFEDelq121LoanCount12M', 
'SBFEDelq121LoanCount24M', 
'SBFEDelq121LoanCount36M', 
'SBFEDelq121LoanCount60M', 
'SBFEDelq121LoanCount84M', 
'SBFEDelq121LoanCountEver', 
'SBFEDelq121LineCount', 
'SBFEDelq121LineCount06M', 
'SBFEDelq121LineCount12M', 
'SBFEDelq121LineCount24M', 
'SBFEDelq121LineCount36M', 
'SBFEDelq121LineCount60M', 
'SBFEDelq121LineCount84M', 
'SBFEDelq121LineCountEver', 
'SBFEDelq121CardCount', 
'SBFEDelq121CardCount06M', 
'SBFEDelq121CardCount12M', 
'SBFEDelq121CardCount24M', 
'SBFEDelq121CardCount36M', 
'SBFEDelq121CardCount60M', 
'SBFEDelq121CardCount84M', 
'SBFEDelq121CardCountEver', 
'SBFEDelq121LeaseCount', 
'SBFEDelq121LeaseCount06M', 
'SBFEDelq121LeaseCount12M', 
'SBFEDelq121LeaseCount24M', 
'SBFEDelq121LeaseCount36M', 
'SBFEDelq121LeaseCount60M', 
'SBFEDelq121LeaseCount84M', 
'SBFEDelq121LeaseCountEver', 
'SBFEDelq121LetterCount', 
'SBFEDelq121LetterCount06M', 
'SBFEDelq121LetterCount12M', 
'SBFEDelq121LetterCount24M', 
'SBFEDelq121LetterCount36M', 
'SBFEDelq121LetterCount60M', 
'SBFEDelq121LetterCount84M', 
'SBFEDelq121LetterCountEver', 
'SBFEDelq121OELineCount', 
'SBFEDelq121OELineCount06M', 
'SBFEDelq121OELineCount12M', 
'SBFEDelq121OELineCount24M', 
'SBFEDelq121OELineCount36M', 
'SBFEDelq121OELineCount60M', 
'SBFEDelq121OELineCount84M', 
'SBFEDelq121OELineCountEver', 
'SBFEDelq121OtherCount', 
'SBFEDelq121OtherCount06M', 
'SBFEDelq121OtherCount12M', 
'SBFEDelq121OtherCount24M', 
'SBFEDelq121OtherCount36M', 
'SBFEDelq121OtherCount60M', 
'SBFEDelq121OtherCount84M', 
'SBFEDelq121OtherCountEver', 
'SBFEDelq31Ratio', 
'SBFEDelq31LoanRatio', 
'SBFEDelq31LineRatio', 
'SBFEDelq31CardRatio', 
'SBFEDelq31LeaseRatio', 
'SBFEDelq31LetterRatio', 
'SBFEDelq31OELineRatio', 
'SBFEDelq31OtherRatio', 
'SBFEDelq61Ratio', 
'SBFEDelq61LoanRatio', 
'SBFEDelq61LineRatio', 
'SBFEDelq61CardRatio', 
'SBFEDelq61LeaseRatio', 
'SBFEDelq61LetterRatio', 
'SBFEDelq61OELineRatio', 
'SBFEDelq61OtherRatio', 
'SBFEDelq91Ratio', 
'SBFEDelq91LoanRatio', 
'SBFEDelq91LineRatio', 
'SBFEDelq91CardRatio', 
'SBFEDelq91LeaseRatio', 
'SBFEDelq91LetterRatio', 
'SBFEDelq91OELineRatio', 
'SBFEDelq91OtherRatio', 
'SBFEDelq121Ratio', 
'SBFEDelq121LoanRatio', 
'SBFEDelq121LineRatio', 
'SBFEDelq121CardRatio', 
'SBFEDelq121LeaseRatio', 
'SBFEDelq121LetterRatio', 
'SBFEDelq121OELineRatio', 
'SBFEDelq121OtherRatio', 
'SBFEDelq1OccurCount06M', 
'SBFEDelq1OccurCount12M', 
'SBFEDelq1OccurCount24M', 
'SBFEDelq1OccurCount36M', 
'SBFEDelq1OccurCount60M', 
'SBFEDelq1OccurCount84M', 
'SBFEDelq1OccurCountEver', 
'SBFEDelq31OccurCount06M', 
'SBFEDelq31OccurCount12M', 
'SBFEDelq31OccurCount24M', 
'SBFEDelq31OccurCount36M', 
'SBFEDelq31OccurCount60M', 
'SBFEDelq31OccurCount84M', 
'SBFEDelq31OccurCountEver', 
'SBFEDelq31OccurLoanCount06M', 
'SBFEDelq31OccurLoanCount12M', 
'SBFEDelq31OccurLoanCount24M', 
'SBFEDelq31OccurLoanCount36M', 
'SBFEDelq31OccurLoanCount60M', 
'SBFEDelq31OccurLoanCount84M', 
'SBFEDelq31OccurLoanCountEver', 
'SBFEDelq31OccurLineCount06M', 
'SBFEDelq31OccurLineCount12M', 
'SBFEDelq31OccurLineCount24M', 
'SBFEDelq31OccurLineCount36M', 
'SBFEDelq31OccurLineCount60M', 
'SBFEDelq31OccurLineCount84M', 
'SBFEDelq31OccurLineCountEver', 
'SBFEDelq31OccurCardCount06M', 
'SBFEDelq31OccurCardCount12M', 
'SBFEDelq31OccurCardCount24M', 
'SBFEDelq31OccurCardCount36M', 
'SBFEDelq31OccurCardCount60M', 
'SBFEDelq31OccurCardCount84M', 
'SBFEDelq31OccurCardCountEver', 
'SBFEDelq31OccurLeaseCount06M', 
'SBFEDelq31OccurLeaseCount12M', 
'SBFEDelq31OccurLeaseCount24M', 
'SBFEDelq31OccurLeaseCount36M', 
'SBFEDelq31OccurLeaseCount60M', 
'SBFEDelq31OccurLeaseCount84M', 
'SBFEDelq31OccurLeaseCountEver', 
'SBFEDelq31OccurLetterCount06M', 
'SBFEDelq31OccurLetterCount12M', 
'SBFEDelq31OccurLetterCount24M', 
'SBFEDelq31OccurLetterCount36M', 
'SBFEDelq31OccurLetterCount60M', 
'SBFEDelq31OccurLetterCount84M', 
'SBFEDelq31OccurLetterCountEver', 
'SBFEDelq31OccurOELineCount06M', 
'SBFEDelq31OccurOELineCount12M', 
'SBFEDelq31OccurOELineCount24M', 
'SBFEDelq31OccurOELineCount36M', 
'SBFEDelq31OccurOELineCount60M', 
'SBFEDelq31OccurOELineCount84M', 
'SBFEDelq31OccurOELineCountEver', 
'SBFEDelq31OccurOtherCount06M', 
'SBFEDelq31OccurOtherCount12M', 
'SBFEDelq31OccurOtherCount24M', 
'SBFEDelq31OccurOtherCount36M', 
'SBFEDelq31OccurOtherCount60M', 
'SBFEDelq31OccurOtherCount84M', 
'SBFEDelq31OccurOtherCountEver', 
'SBFEDelq61OccurCount06M', 
'SBFEDelq61OccurCount12M', 
'SBFEDelq61OccurCount24M', 
'SBFEDelq61OccurCount36M', 
'SBFEDelq61OccurCount60M', 
'SBFEDelq61OccurCount84M', 
'SBFEDelq61OccurCountEver', 
'SBFEDelq61OccurLoanCount06M', 
'SBFEDelq61OccurLoanCount12M', 
'SBFEDelq61OccurLoanCount24M', 
'SBFEDelq61OccurLoanCount36M', 
'SBFEDelq61OccurLoanCount60M', 
'SBFEDelq61OccurLoanCount84M', 
'SBFEDelq61OccurLoanCountEver', 
'SBFEDelq61OccurLineCount06M', 
'SBFEDelq61OccurLineCount12M', 
'SBFEDelq61OccurLineCount24M', 
'SBFEDelq61OccurLineCount36M', 
'SBFEDelq61OccurLineCount60M', 
'SBFEDelq61OccurLineCount84M', 
'SBFEDelq61OccurLineCountEver', 
'SBFEDelq61OccurCardCount06M', 
'SBFEDelq61OccurCardCount12M', 
'SBFEDelq61OccurCardCount24M', 
'SBFEDelq61OccurCardCount36M', 
'SBFEDelq61OccurCardCount60M', 
'SBFEDelq61OccurCardCount84M', 
'SBFEDelq61OccurCardCountEver', 
'SBFEDelq61OccurLeaseCount06M', 
'SBFEDelq61OccurLeaseCount12M', 
'SBFEDelq61OccurLeaseCount24M', 
'SBFEDelq61OccurLeaseCount36M', 
'SBFEDelq61OccurLeaseCount60M', 
'SBFEDelq61OccurLeaseCount84M', 
'SBFEDelq61OccurLeaseCountEver', 
'SBFEDelq61OccurLetterCount06M', 
'SBFEDelq61OccurLetterCount12M', 
'SBFEDelq61OccurLetterCount24M', 
'SBFEDelq61OccurLetterCount36M', 
'SBFEDelq61OccurLetterCount60M', 
'SBFEDelq61OccurLetterCount84M', 
'SBFEDelq61OccurLetterCountEver', 
'SBFEDelq61OccurOELineCount06M', 
'SBFEDelq61OccurOELineCount12M', 
'SBFEDelq61OccurOELineCount24M', 
'SBFEDelq61OccurOELineCount36M', 
'SBFEDelq61OccurOELineCount60M', 
'SBFEDelq61OccurOELineCount84M', 
'SBFEDelq61OccurOELineCountEver', 
'SBFEDelq61OccurOtherCount06M', 
'SBFEDelq61OccurOtherCount12M', 
'SBFEDelq61OccurOtherCount24M', 
'SBFEDelq61OccurOtherCount36M', 
'SBFEDelq61OccurOtherCount60M', 
'SBFEDelq61OccurOtherCount84M', 
'SBFEDelq61OccurOtherCountEver', 
'SBFEDelq91OccurCount06M', 
'SBFEDelq91OccurCount12M', 
'SBFEDelq91OccurCount24M', 
'SBFEDelq91OccurCount36M', 
'SBFEDelq91OccurCount60M', 
'SBFEDelq91OccurCount84M', 
'SBFEDelq91OccurCountEver', 
'SBFEDelq91OccurLoanCount06M', 
'SBFEDelq91OccurLoanCount12M', 
'SBFEDelq91OccurLoanCount24M', 
'SBFEDelq91OccurLoanCount36M', 
'SBFEDelq91OccurLoanCount60M', 
'SBFEDelq91OccurLoanCount84M', 
'SBFEDelq91OccurLoanCountEver', 
'SBFEDelq91OccurLineCount06M', 
'SBFEDelq91OccurLineCount12M', 
'SBFEDelq91OccurLineCount24M', 
'SBFEDelq91OccurLineCount36M', 
'SBFEDelq91OccurLineCount60M', 
'SBFEDelq91OccurLineCount84M', 
'SBFEDelq91OccurLineCountEver', 
'SBFEDelq91OccurCardCount06M', 
'SBFEDelq91OccurCardCount12M', 
'SBFEDelq91OccurCardCount24M', 
'SBFEDelq91OccurCardCount36M', 
'SBFEDelq91OccurCardCount60M', 
'SBFEDelq91OccurCardCount84M', 
'SBFEDelq91OccurCardCountEver', 
'SBFEDelq91OccurLeaseCount06M', 
'SBFEDelq91OccurLeaseCount12M', 
'SBFEDelq91OccurLeaseCount24M', 
'SBFEDelq91OccurLeaseCount36M', 
'SBFEDelq91OccurLeaseCount60M', 
'SBFEDelq91OccurLeaseCount84M', 
'SBFEDelq91OccurLeaseCountEver', 
'SBFEDelq91OccurLetterCount06M', 
'SBFEDelq91OccurLetterCount12M', 
'SBFEDelq91OccurLetterCount24M', 
'SBFEDelq91OccurLetterCount36M', 
'SBFEDelq91OccurLetterCount60M', 
'SBFEDelq91OccurLetterCount84M', 
'SBFEDelq91OccurLetterCountEver', 
'SBFEDelq91OccurOELineCount06M', 
'SBFEDelq91OccurOELineCount12M', 
'SBFEDelq91OccurOELineCount24M', 
'SBFEDelq91OccurOELineCount36M', 
'SBFEDelq91OccurOELineCount60M', 
'SBFEDelq91OccurOELineCount84M', 
'SBFEDelq91OccurOELineCountEver', 
'SBFEDelq91OccurOtherCount06M', 
'SBFEDelq91OccurOtherCount12M', 
'SBFEDelq91OccurOtherCount24M', 
'SBFEDelq91OccurOtherCount36M', 
'SBFEDelq91OccurOtherCount60M', 
'SBFEDelq91OccurOtherCount84M', 
'SBFEDelq91OccurOtherCountEver', 
'SBFEDelq121OccurCount06M', 
'SBFEDelq121OccurCount12M', 
'SBFEDelq121OccurCount24M', 
'SBFEDelq121OccurCount36M', 
'SBFEDelq121OccurCount60M', 
'SBFEDelq121OccurCount84M', 
'SBFEDelq121OccurCountEver', 
'SBFEDelq121OccurLoanCount06M', 
'SBFEDelq121OccurLoanCount12M', 
'SBFEDelq121OccurLoanCount24M', 
'SBFEDelq121OccurLoanCount36M', 
'SBFEDelq121OccurLoanCount60M', 
'SBFEDelq121OccurLoanCount84M', 
'SBFEDelq121OccurLoanCountEver', 
'SBFEDelq121OccurLineCount06M', 
'SBFEDelq121OccurLineCount12M', 
'SBFEDelq121OccurLineCount24M', 
'SBFEDelq121OccurLineCount36M', 
'SBFEDelq121OccurLineCount60M', 
'SBFEDelq121OccurLineCount84M', 
'SBFEDelq121OccurLineCountEver', 
'SBFEDelq121OccurCardCount06M', 
'SBFEDelq121OccurCardCount12M', 
'SBFEDelq121OccurCardCount24M', 
'SBFEDelq121OccurCardCount36M', 
'SBFEDelq121OccurCardCount60M', 
'SBFEDelq121OccurCardCount84M', 
'SBFEDelq121OccurCardCountEver', 
'SBFEDelq121OccurLeaseCount06M', 
'SBFEDelq121OccurLeaseCount12M', 
'SBFEDelq121OccurLeaseCount24M', 
'SBFEDelq121OccurLeaseCount36M', 
'SBFEDelq121OccurLeaseCount60M', 
'SBFEDelq121OccurLeaseCount84M', 
'SBFEDelq121OccurLeaseCountEver', 
'SBFEDelq121OccurLetterCount06M', 
'SBFEDelq121OccurLetterCount12M', 
'SBFEDelq121OccurLetterCount24M', 
'SBFEDelq121OccurLetterCount36M', 
'SBFEDelq121OccurLetterCount60M', 
'SBFEDelq121OccurLetterCount84M', 
'SBFEDelq121OccurLetterCountEver', 
'SBFEDelq121OccurOELineCount06M', 
'SBFEDelq121OccurOELineCount12M', 
'SBFEDelq121OccurOELineCount24M', 
'SBFEDelq121OccurOELineCount36M', 
'SBFEDelq121OccurOELineCount60M', 
'SBFEDelq121OccurOELineCount84M', 
'SBFEDelq121OccurOELineCountEver', 
'SBFEDelq121OccurOtherCount06M', 
'SBFEDelq121OccurOtherCount12M', 
'SBFEDelq121OccurOtherCount24M', 
'SBFEDelq121OccurOtherCount36M', 
'SBFEDelq121OccurOtherCount60M', 
'SBFEDelq121OccurOtherCount84M', 
'SBFEDelq121OccurOtherCountEver', 
'SBFEDelqAmt', 
'SBFEDelq01Amt', 
'SBFEDelq31Amt', 
'SBFEDelq61Amt', 
'SBFEDelq91Amt', 
'SBFEDelq121Amt', 
'SBFEDelqLoanAmt', 
'SBFEDelq01LoanAmt', 
'SBFEDelq31LoanAmt', 
'SBFEDelq61LoanAmt', 
'SBFEDelq91LoanAmt', 
'SBFEDelq121LoanAmt', 
'SBFEDelqLineAmt', 
'SBFEDelq01LineAmt', 
'SBFEDelq31LineAmt', 
'SBFEDelq61LineAmt', 
'SBFEDelq91LineAmt', 
'SBFEDelq121LineAmt', 
'SBFEDelqCardAmt', 
'SBFEDelq01CardAmt', 
'SBFEDelq31CardAmt', 
'SBFEDelq61CardAmt', 
'SBFEDelq91CardAmt', 
'SBFEDelq121CardAmt', 
'SBFEDelqLeaseAmt', 
'SBFEDelq01LeaseAmt', 
'SBFEDelq31LeaseAmt', 
'SBFEDelq61LeaseAmt', 
'SBFEDelq91LeaseAmt', 
'SBFEDelq121LeaseAmt', 
'SBFEDelqLetterAmt', 
'SBFEDelq01LetterAmt', 
'SBFEDelq31LetterAmt', 
'SBFEDelq61LetterAmt', 
'SBFEDelq91LetterAmt', 
'SBFEDelq121LetterAmt', 
'SBFEDelqOELineAmt', 
'SBFEDelq01OELineAmt', 
'SBFEDelq31OELineAmt', 
'SBFEDelq61OELineAmt', 
'SBFEDelq91OELineAmt', 
'SBFEDelq121OELineAmt', 
'SBFEDelqOtherAmt', 
'SBFEDelq01OtherAmt', 
'SBFEDelq31OtherAmt', 
'SBFEDelq61OtherAmt', 
'SBFEDelq91OtherAmt', 
'SBFEDelq121OtherAmt', 
'SBFEDelqAvg06M', 
'SBFEDelqAvgAmt12M', 
'SBFEDelqAvgAmt24M', 
'SBFEDelqAvgAmt36M', 
'SBFEDelqAvgAmt60M', 
'SBFEDelqAvgAmt84M', 
'SBFEDelqAvgAmtEver', 
'SBFEChargeoffCount', 
'SBFEChargeoffLoanCount', 
'SBFEChargeoffLineCount', 
'SBFEChargeoffCardCount', 
'SBFEChargeoffLeaseCount', 
'SBFEChargeoffLetterCount', 
'SBFEChargeoffOELineCount', 
'SBFEChargeoffOtherCount', 
'SBFEChargeoffAmt', 
'SBFEChargeoffLoanAmt', 
'SBFEChargeoffLineAmt', 
'SBFEChargeoffCardAmt', 
'SBFEChargeoffLeaseAmt', 
'SBFEChargeoffLetterAmt', 
'SBFEChargeoffOELineAmt', 
'SBFEChargeoffOtherAmt', 
'SBFETimeNewestChargeoff', 
'SBFEBalloonCount', 
'SBFEBalloonPaymentAmt', 
'SBFEBalloonPaymentDueDate', 
'SBFELegalStructure', 
'SBFESICCode', 
'SBFENAICSCode', 
'SBFEGovGuaranteeCount', 
'SBFEGuarantorAccountCount', 
'SBFEGuarantorMinCount', 
'SBFEGuarantorMaxCount', 
'SBFEPrincipalAccountCount', 
'SBFEPrincipalMinCount', 
'SBFEPrincipalMaxCount', 
'Model_1_Name', 
'Model_1_Score', 
'Model_1_RC1', 
'Model_1_RC2', 
'Model_1_RC3', 
'Model_1_RC4', 
'Model_2_Name', 
'Model_2_Score', 
'Model_2_RC1', 
'Model_2_RC2', 
'Model_2_RC3', 
'Model_2_RC4', 
]